<img src="./ESSENCE.png" width=700px>

### Reflection pattern

We generate a response with our first query and then add this content to the request for a critique or further refinement.

The first request may be a standard request as in this case to generate some Python Code and the system prompt reflects this.

Then with the returned content, we create a second request with the content included, (RAG), with some instruction to critique it or refine it.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-TV


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)

Certainly! Below is a Python implementation that demonstrates how to make a GET request to an API using the Requests library. This implementation includes error handling and proper responses management.

```python
import requests

def fetch_api_data(url):
    """
    Fetches data from the provided API URL.

    Parameters:
    url (str): The URL of the API endpoint.

    Returns:
    dict: The JSON response from the API if the request was successful.
    None: If the request failed or data could not be retrieved.
    """
    try:
        # Making a GET request to the API
        response = requests.get(url)

        # Raise an HTTPError for bad responses (4xx or 5xx)
        response.raise_for_status()

        # Attempt to parse the response as JSON
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    except ValueError as e:
        print(f"Error parsing JSON: {e}")

    return None

if __name__ == "__main__":
    # Example usage
    api_url = "https://api.example.com/data"
    data = fetch_api_data(api_url)

    if data:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data.")
```

### Explanation:
1. **Imports**: The `requests` module is imported which is essential for making HTTP requests.
2. **Function `fetch_api_data`**: 
   - Takes a URL as input and makes a GET request.
   - It uses `response.raise_for_status()` to handle HTTP errors gracefully.
   - Parses the response to JSON and returns it.
   - Catches exceptions from network issues and JSON parsing errors, printing error messages.
3. **Main Block**: An example of how to use the function. It specifies an API URL and prints the fetched data or an error message.

### Notes:
- Make sure to replace `https://api.example.com/data` with the actual API endpoint you want to call.
- To run this code, you need to have the `requests` library installed. You can install it via pip:
  ```
  pip install requests
  ``` 

Feel free to modify the URL and add any headers or parameters as needed for your specific API request!

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)

Your implementation of the API GET request using the Requests library is quite solid. It includes essential functionalities such as error handling and JSON parsing, which are crucial for working with web APIs. Here are a few critiques and recommendations that could enhance your code further:

### Critiques:

1. **Error Handling Messages**: The error messages currently print directly to the console. In a production environment, it might be better to log these errors using Python’s built-in `logging` module, which allows for better control over logging levels and output destinations.

2. **Return Value Consistency**: The function can return `None` if an exception is raised, but this can be ambiguous with a successful response that may just happen to be `None`. It may be helpful to consider raising a custom exception or returning a tuple `(data, error)` for better clarity on the outcome.

3. **Function Documentation**: The docstring is good, but it could specify what type of errors might occur or what specific kind of data the function is expected to return. This information can be helpful for users of the function.

4. **User-Agent Header**: It’s often advisable to include a User-Agent header in your requests, as some APIs might block requests without it. 

5. **Use of `ValueError` Exception**: Catching `ValueError` specifically might not be necessary since `json()` will raise `requests.exceptions.JSONDecodeError`, which is derived from `ValueError`. You could handle that error directly instead.

### Recommendations:

1. **Enhance Logging**: Consider using Python's built-in logging module for better logging:
   ```python
   import logging

   logging.basicConfig(level=logging.INFO)

   def fetch_api_data(url):
       logger = logging.getLogger(__name__)
       ...
       except requests.exceptions.RequestException as e:
           logger.error("An error occurred: %s", e)
       except requests.exceptions.JSONDecodeError as e:
           logger.error("Error parsing JSON: %s", e)
   ```

2. **Custom Exceptions**: If you want to enhance error handling, consider implementing a custom exception for better clarity, like this:
   ```python
   class APIError(Exception):
       pass

   def fetch_api_data(url):
       try:
           ...
       except requests.exceptions.RequestException as e:
           raise APIError(f"API request failed: {e}")
   ```

3. **User-Agent Header**: Example:
   ```python
   headers = {
       'User-Agent': 'YourAppName/1.0'
   }
   response = requests.get(url, headers=headers)
   ```

4. **Parameterization of URL**: If you need to pass parameters to your API call, consider expanding your function to accept query parameters:
   ```python
   def fetch_api_data(url, params=None):
       response = requests.get(url, headers=headers, params=params)
   ```

5. **Timeouts for Requests**: Adding a timeout to your requests can prevent the program from hanging indefinitely. Consider adding:
   ```python
   response = requests.get(url, headers=headers, timeout=5)  # 5 seconds timeout
   ```

6. **Testing and Validating Responses**: Checking if the response is indeed JSON can further solidify the reliability of your function:
   ```python
   if response.headers.get('Content-Type') == 'application/json':
       return response.json()
   ```

### Example of Enhanced Code:
Here’s a refined version of your function incorporating some of these recommendations:

```python
import requests
import logging

logging.basicConfig(level=logging.INFO)

class APIError(Exception):
    pass

def fetch_api_data(url, params=None):
    """
    Fetches data from the provided API URL.

    Parameters:
    url (str): The URL of the API endpoint.
    params (dict): Optional query parameters to include in the request.

    Returns:
    dict: The JSON response from the API if the request was successful.
    Raises APIError: If the request failed or data could not be retrieved.
    """
    headers = {
        'User-Agent': 'YourAppName/1.0'
    }
    try:
        response = requests.get(url, headers=headers, params=params, timeout=5)
        response.raise_for_status()

        if response.headers.get('Content-Type') == 'application/json':
            return response.json()
        else:
            raise APIError(f"Unexpected content type: {response.headers.get('Content-Type')}")

    except requests.exceptions.RequestException as e:
        logging.error("An error occurred: %s", e)
        raise APIError(f"API request failed: {e}")

if __name__ == "__main__":
    api_url = "https://api.example.com/data"
    try:
        data = fetch_api_data(api_url)
        print("Data fetched successfully:")
        print(data)
    except APIError:
        print("Failed to fetch data.")
```

This enhanced version adheres to best practices and improves the robustness of your API fetching process. Great job, and keep up the good work!

Add CRITIQUE to chat...


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for your insightful critiques and recommendations! Based on your feedback, I've revised the implementation to address the points you've raised, incorporating enhanced error handling, logging, user-agent headers, and optional parameters. Here’s the improved version of the Python implementation using the Requests library for making API requests:

```python
import requests
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

class APIError(Exception):
    """Custom exception for API errors."""
    pass

def fetch_api_data(url, params=None):
    """
    Fetches data from the provided API URL.

    Parameters:
    url (str): The URL of the API endpoint.
    params (dict, optional): Query parameters to include in the request.

    Returns:
    dict: The JSON response from the API if the request was successful.
    
    Raises:
    APIError: If the request failed or data could not be retrieved.
    """
    headers = {
        'User-Agent': 'YourAppName/1.0'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params, timeout=5)
        response.raise_for_status()

        # Check if the response content type is JSON
        if response.headers.get('Content-Type') == 'application/json':
            return response.json()
        else:
            raise APIError(f"Unexpected content type: {response.headers.get('Content-Type')}")

    except requests.exceptions.RequestException as e:
        logging.error("An error occurred: %s", e)
        raise APIError(f"API request failed: {e}")
    except requests.exceptions.JSONDecodeError as e:
        logging.error("Error parsing JSON: %s", e)
        raise APIError(f"Failed to decode JSON: {e}")

if __name__ == "__main__":
    # Example usage
    api_url = "https://api.example.com/data"
    try:
        data = fetch_api_data(api_url)
        print("Data fetched successfully:")
        print(data)
    except APIError:
        print("Failed to fetch data.")
```

### Key Improvements:
1. **Logging**: Using Python’s built-in logging module instead of print statements for better tracking of errors.
2. **Custom Exception**: Introduced a custom `APIError` exception to convey API-related issues more clearly.
3. **User-Agent Header**: Included a User-Agent header in the requests.
4. **Parameterization**: The `params` argument allows for query parameters to be passed.
5. **Timeout Handling**: Added a timeout to prevent indefinite hanging on requests.
6. **Content-Type Check**: Validates the content type of the response before processing it.

This version aims to create a more robust and maintainable API client while adhering to best practices in Python programming. Thank you for your feedback, and please let me know if you have any more suggestions or need further assistance!

<br>

## We can of course make this a Class...
